## 背景
- 注册了很多个邮箱，而每个邮箱又注册了很多个网站。清理这些账号时特别麻烦。

## 初步方案
- 用 POP3 接收邮件后进行处理，得到邮箱注册过的所有网站

## 待验证问题
- email 的内容应该如何读取？
- 应该输出什么样的排版格式，哪些信息是需要展示的？

## 说明

选择IMAP，不选择POP3，因为POP3不支持摘要浏览，且IMAP支持更多功能特性。

IMAP提供的摘要浏览功能可以让你在阅读完所有的邮件到达时间、主题、发件人、大小等信息后才作出是否下载的决定

**谷歌邮箱2025年开始不再支持**: 自 2025 年 1 月起，“启用 IMAP”和“停用 IMAP”选项将无法再使用。Gmail 中的 IMAP 访问功能始终处于启用状态，您当前与其他电子邮件客户端的连接不会受到影响。您无需采取任何行动
> https://support.google.com/mail/answer/7126229?hl=zh-Hans&ref_topic=7280141&sjid=9501973201011786493-NA

## 步骤

1. 登录IMAP服务器
2. 获取所有邮件的主题、发件人、时间信息
3. 存储到sqlite数据库中（可选）
4. 对发件人字段进行去重
5. 输出文本，将去重后的发件人信息输出到文本文件中

In [ ]:
# 登录IMAP服务器
from imapclient import IMAPClient
import email
from email.header import decode_header
from email.utils import parseaddr

username = '替换成你的邮件名，包含@全文'
password = '替换成你的授权码'
# 网易云邮箱开头IMAP服务可参考：https://help.mail.163.com/faqDetail.do?code=d7a5dc8471cd0c0e8b4b8f4f8e49998b374173cfe9171305fa1ce630d7f67ac2a5feb28b66796d3b

server = IMAPClient("imap.163.com", ssl=True, port=993)
server.login(username, password)
# 解决网易服务器报错：('NO', [b'SELECT Unsafe Login. Please contact kefu@188.com for help'])
# 添加id即可。参考：https://github.com/OfflineIMAP/offlineimap/issues/696
server.id_({"name": "IMAPClient", "version": "2.1.0"})

folder_info = server.select_folder('INBOX', readonly=False)
print(f"文件夹中共有 {folder_info[b'EXISTS']} 封邮件")


In [ ]:
# 获取所有邮件的信息并关闭连接
messages = server.search(['ALL'])
print(f"server.search(['ALL']) 命令：找到 {len(messages)} 条邮件id")
envelopes = server.fetch(messages, ['ENVELOPE'])
server.logout()

In [ ]:
# 创建或打开sqlite数据库
import sqlite3
import os

# 数据库位置
DATABASE_PATH = 'email_metadata.db'

def create_database():
    """创建数据库和表结构"""
    conn = sqlite3.connect(DATABASE_PATH)
    try:
        cursor = conn.cursor()
        # 创建邮件元数据表
        cursor.execute('''
            CREATE TABLE IF NOT EXISTS emails (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                message_id INTEGER NOT NULL UNIQUE,
                sender_email TEXT NOT NULL,
                subject TEXT,
                send_time DATETIME,
                created_at DATETIME DEFAULT CURRENT_TIMESTAMP
            )
        ''')
        # 创建索引加速查询
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_sender ON emails (sender_email)')
        cursor.execute('CREATE INDEX IF NOT EXISTS idx_send_time ON emails (send_time)')
        conn.commit()
    finally:
        conn.close()

if not os.path.exists(DATABASE_PATH):
    create_database()

In [ ]:
# 规整数据成列表
datas = []
for msg_id, data in envelopes.items():
    envelope = data[b'ENVELOPE']
    
    # 解析发件人
    sender = envelope.from_[0]  # 取第一个发件人
    sender_email = f"{sender.mailbox.decode()}@{sender.host.decode()}"
    
    # 解析主题（自动解码编码内容）
    subject, encoding = decode_header(envelope.subject.decode())[0]
    if encoding:
        subject = subject.decode(encoding)
    else:
        subject = subject.decode('utf-8', errors='ignore')
    
    # 解析日期（转换为本地时间）
    date_str = envelope.date
    local_time = date_str
    
    datas.append([msg_id, sender_email, subject, local_time])
print(f"列表数据长度：{len(datas)}")


In [ ]:
# 更新数据到数据时中
def save_to_database(envelopes):
    """将数据存入数据库"""
    conn = sqlite3.connect(DATABASE_PATH)
    try:
        cursor = conn.cursor()
        # 使用事务批量插入
        insert_count = 0
        cursor.execute('BEGIN TRANSACTION')
        
        for data in datas:
            try:
                cursor.execute('''
                    INSERT OR IGNORE INTO emails 
                    (message_id, sender_email, subject, send_time)
                    VALUES (?, ?, ?, ?)
                ''', (data[0], data[1], data[2], data[3]))
                insert_count += 1
            except sqlite3.IntegrityError as e:
                print(f"跳过重复邮件 ID {msg_id}: {str(e)}")
        
        conn.commit()
        print(f"成功插入 {insert_count} 条记录")
    except Exception as e:
        conn.rollback()
        print(f"数据库操作失败: {str(e)}")
    finally:
        conn.close()

save_to_database(envelopes)

In [ ]:
# 查询SQL 输出数据

def execute_and_format_query(query, params=()):
    """执行SQL查询并返回格式化结果"""
    conn = sqlite3.connect(DATABASE_PATH)
    try:
        conn.row_factory = sqlite3.Row  # 使用字典式结果
        cursor = conn.cursor()
        
        # 执行查询
        cursor.execute(query, params)
        results = cursor.fetchall()
        
        if not results:
            return "没有查询到匹配结果"
            
        # 获取字段名称
        columns = [col[0] for col in cursor.description]
        
        # 生成格式化文本
        output = []
        for idx, row in enumerate(results, 1):
            line = []
            for col in columns:
                # 处理空值情况
                value = row[col] if row[col] is not None else "NULL"
                # 格式化为 "字段: 值"
                # line.append(f"{col}: {value}")
                line.append(str(value))
            # 添加行号并组合字段
            output.append(f"[[{line[0]}]]")
            #output.append(f"[记录 {idx}] " + " | ".join(line))
            
        # 添加统计信息
        # output.append(f"\n共找到 {len(results)} 条记录，包含 {len(columns)} 个字段")
        return "\n".join(output)
        
    except sqlite3.Error as e:
        return f"查询执行失败: {str(e)}"
    finally:
        conn.close()

# 使用示例
query = """
SELECT DISTINCT SUBSTR(sender_email, INSTR(sender_email, '@') + 1) AS domain
  FROM emails
 WHERE INSTR(sender_email, '@') > 0
 ORDER BY domain;
"""

output_to_print = f"# {username}\n"
output_to_print += execute_and_format_query(query)

print(output_to_print)